In [ ]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gradio as gr
import openai
import re

In [ ]:
# Set up OpenAI API credentials
openai.api_key = "sk your open ai key"

In [ ]:
# Define the prompt
prompt = """
You are an SQL expert and you want to help users with their database queries. Your job is to read a database structure given by the user "structure", and a question given by the user "question", and to return a working SQL query. 

Database Structure example :
Table: Customers
Columns: customer_id (integer), customer_name (string), customer_email (string), customer_phone (string), customer_address (string)

Table: Orders
Columns: order_id (integer), customer_id (integer), order_date (date), total_amount (decimal)

Table: Products
Columns: product_id (integer), product_name (string), price (decimal)

Table: Order_Items
Columns: order_item_id (integer), order_id (integer), product_id (integer), quantity (integer)

You need to write SQL queries to respond to user questions. Each question will be provided in plain English or French. Your task is to generate a working SQL query based on the user question.

Query examples :

User: "Can you retrieve the total number of customers in the database?"
200 "SELECT COUNT(*) FROM Customers"

User: "Quels sont les produits commandés par le client avec l'ID 123?"
200 "SELECT p.product_name FROM Products p INNER JOIN Order_Items oi ON p.product_id = oi.product_id INNER JOIN Orders o ON oi.order_id = o.order_id WHERE o.customer_id = 123"

User: "What is the total revenue generated by the customer with the ID 456?"
200 "SELECT SUM(p.price * oi.quantity) AS total_revenue FROM Products p INNER JOIN Order_Items oi ON p.product_id = oi.product_id INNER JOIN Orders o ON oi.order_id = o.order_id WHERE o.customer_id = 456";

User: "Pouvez-vous afficher le nom et l'adresse de tous les clients?
200 "SELECT customer_name, customer_address FROM Customers"

User: "How many orders were placed on a specific date?"
200 "SELECT COUNT(*) FROM Orders WHERE order_date = '2023-05-01'"

User: "what is the total distance between the moon and earth?"
400

Rules:

- The queries must be correct and clear and should work with the suggested database structure
- Make the queries simple and correct. and the response contain nothing more than the query itself. 
- If the question can be answered with an SQL query using the given database structure, respond with the SQL query.
- If you dont understand the question or believe the user is asking for elements outside of the existing database and queries, respond with "400". just 400, nothing more.
- Don't add or imagine solutions to elements that dont exist. 

Generate the SQL Query. 

"""

In [ ]:
# Function to generate the SQL query
def generate_sql_query(question):

    # Create the prompt with the user's question
    full_prompt = prompt + f"\n \"{question}\" \n"

    # Generate the completion using OpenAI API
    completion = openai.Completion.create(
        engine="text-davinci-003",
        prompt=full_prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.6,
    )

    # Process the first completion choice response
    choice = completion.choices[0]
    print(choice)
    match1 = re.search(r'200 \"(.*?)\"', choice.text)
    match2 = re.search(r'400 \"(.*?)\"', choice.text)
    print(match1)
    print(match2)
    if match1:
        # Extract the SQL query without the response code and clean the response
        response = match1.group(1)  # Extract everything between the double quotes
        return response if response else "Unable to generate the SQL query."
    elif match2:
        # Return an error message
        return "Kindly reformulate your question, either out of context or too complex to translate into a query."

    # Return a generic error message if no valid response is found
    return "Kindly reformulate your question, either out of context or too complex to translate into a query."

quel est le montant global des commandes de l'utilisateur 45

---



In [ ]:
# Create a Gradio interface
def query_interface(question):
    sql_query = generate_sql_query(question)
    print(f"Question: {question}\nSQL Query: {sql_query}\n")  # Print response for debugging
    return sql_query

iface = gr.Interface(fn=query_interface, inputs="text", outputs="text")
iface.launch(inline=True, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7eabe54aa315a3caa2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "200 \"SELECT SUM(oi.quantity * p.price) AS total_amount FROM Order_Items oi INNER JOIN Products p ON oi.product_id = p.product_id INNER JOIN Orders o ON oi.order_id = o.order_id WHERE o.customer_id = 45\""
}
<re.Match object; span=(0, 202), match='200 "SELECT SUM(oi.quantity * p.price) AS total_a>
None
Question: quel est le montant global des commandes de l'utilisateur 45
SQL Query: SELECT SUM(oi.quantity * p.price) AS total_amount FROM Order_Items oi INNER JOIN Products p ON oi.product_id = p.product_id INNER JOIN Orders o ON oi.order_id = o.order_id WHERE o.customer_id = 45

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "400"
}
None
None
Question: quel est le mon nom ?
SQL Query: Kindly reformulate your question, either out of context or too complex to translate into a query.

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "text": "400"
}
None
None
Question: quel est